In [1]:
# import packages needed for scraping

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException

import pandas as pd
import random
import time 
import re

In [4]:
# create path to geckodriver
path = "C:/Users/jpalumbo/geckodriver"

# initialize the webdriver
options = webdriver.FirefoxOptions()

# assign webdriver path and webdriver options as driver
driver = webdriver.Firefox(executable_path=path, options=options)

# keyword or job_title to search for within glassdoor url
keyword = 'data scientist'            
# glassdooor url 
url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword=' + keyword + '&suggestChosen=false&clickSource=searchBox'

# open window with url and search keyword 
driver.get(url)
driver.maximize_window()

In [5]:
# keyphrases and keywords to search for in Job_Description assignment during the scrape

# keyphrase
benefits_keyphrase = ['401 (k)', '401 k', 'paid time off', 'wellness program', 'health savings account', 'health insurance', 'life insurance']
qualifications_keyphrase = ['r studio', 'scikit-learn', 'computer science', 'natural language processing', 'deep learning', 'machine learning', \
                            'big data', 'data mining', 'neural networks', 'artificial intelligence', 'power bi', 'google cloud', \
                            'amazon web services', 'pipeline', 'sql', 'nosql', 'ai/ml']
# keyword
benefits_keyword = ['401(k)', '401k',  'pto', 'dental', 'remote']
qualifications_keyword = ['python', 'r', 'r-studio', 'oracle', 'scala', 'pandas', 'numpy', 'pytorch', 'tensorflow', 'spark', 'tableau'\
                          'statistics', 'math', 'mathematics', 'physics', 'engineering', 'nlp', 'engineer', 'algorithms', \
                          'ml', 'ai', 'scraping', 'classification', 'regression', 'aws', 'api', 'bachelors', 'masters', 'phd']   # add 'flask' next time

In [6]:
# steps to get through opening pages

time.sleep(random.randint(4,8))      # period to wait for page to load

try:
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"span[class='css-8zxfjs']"))).click()  # search for data scientist
except ElementClickInterceptedException:
    pass

try:
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"li[class='react-job-listing css-bkasv9 eigr9kq0']"))).click()  # click on first listing
except ElementClickInterceptedException:
    pass

try:
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"span[class='SVGInline modal_closeIcon']"))).click() # click out of popup window 
except ElementClickInterceptedException:
    pass

In [7]:
jobs = []      # create jobs lsit that stores all scraped job dict info 

In [8]:
location = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[id='sc.location']")))   # locate, clear and enter location to perform job search
location.clear()
location.send_keys('Vermont')    # enter location where desired keyword search should take place
time.sleep(5)

try:
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"span[class='css-8zxfjs']"))).click()  # run the job search
except ElementClickInterceptedException:
    pass 

In [9]:
page_count = 1                    # a count for the number of web pages that have already been scraped, ends once page_count > pages_per_search
count = 1                         # a count for the number of job listings that have already been scraped on current web page, resets every new page
count_str = str(count)            # a string form of the count used to insert into xpath in order to find correct element (job_listing) on page
pages_per_search = 30             # how many pages to scrape per 'job + location' search
jobs_per_page = 30                # how many jobs to scrape per web page

In [10]:
# scrapes and stores the data, IF Glassdoor times out, simply re-run \
# this cell and the script will pick up where it left off
while page_count <= pages_per_search:
    try: 
        while count <= jobs_per_page:
            try: 
                job_listing = driver.find_element_by_xpath("/html/body/div[3]/div/div/div[1]/div/div[2]/section/article/div[1]/ul/li["+ count_str +"]")   # clicks on desired job_listing per page
            except NoSuchElementException: 
                print("\n *****SCRAPING TERMINATED*****")  # terminate scrape if no jobs exist on the page
                break
            time.sleep(random.randint(3,6))
            job_listing.click()                  # click on the desired job listing
            time.sleep(random.randint(3,7))
            
            # open 'Show More' for job_description text so all text can be scraped - this MUST BE DONE otherwise selenium will only scrape what is visible
            show_more = driver.find_element_by_xpath("/html/body/div[3]/div/div/div[1]/div/div[2]/section/div/div/article/div/div[2]/div[1]/div[1]/div/div[2]") 
            show_more.click()                  # click on the 'show more' portion of the page
            time.sleep(random.randint(1,3))

            # grab all desired text from each xpath element
            try:
                try: company_name_rating = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div/div[2]/section/div/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[1]').text
                except NoSuchElementException: company_name_rating = -1 

                try: job_title = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div/div[2]/section/div/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[2]').text
                except NoSuchElementException: job_title = -1 

                try: location = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div/div[2]/section/div/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[3]').text
                except NoSuchElementException: location = -1 

                try: salary_estimate = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div/div[2]/section/div/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[4]/span').text
                except NoSuchElementException: salary_estimate = -1 

                try: company_size = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div/div[2]/section/div/div/article/div/div[2]/div[1]/div[3]/div/div/div[1]/div/div[1]/span[2]').text
                except NoSuchElementException: company_size = -1 

                try: company_type = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div/div[2]/section/div/div/article/div/div[2]/div[1]/div[3]/div/div/div[1]/div/div[3]/span[2]').text
                except NoSuchElementException: company_type = -1 

                try: company_sector = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div/div[2]/section/div/div/article/div/div[2]/div[1]/div[3]/div/div/div[1]/div/div[5]/span[2]').text
                except NoSuchElementException: company_sector = -1 

                try: year_founded = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div/div[2]/section/div/div/article/div/div[2]/div[1]/div[3]/div/div/div[1]/div/div[2]/span[2]').text
                except NoSuchElementException: year_founded = -1 

                try: company_industry = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div/div[2]/section/div/div/article/div/div[2]/div[1]/div[3]/div/div/div[1]/div/div[4]/span[2]').text
                except NoSuchElementException: company_industry = -1 

                try: company_revenue = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div/div[2]/section/div/div/article/div/div[2]/div[1]/div[3]/div/div/div[1]/div/div[6]/span[2]').text
                except NoSuchElementException: company_revenue = -1 

                try: job_description = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div/div[2]/section/div/div/article/div/div[2]/div[1]/div[1]/div/div[1]/div').text
                except NoSuchElementException: job_description = -1 
            except:
                time.sleep(3)     

            ### lists to contain keyphrases and keywords from the job_description text
            benefits_list = []
            qualifications_list = []

            # make all elements in job_description string lowercase
            job_description = job_description.lower()

            # append benefits keyphrases to benefits list
            for i in benefits_keyphrase:
                if i in job_description:
                    benefits_list.append(i)

            # append qualifications keyphrases to qualifications list 
            for i in qualifications_keyphrase:
                if i in job_description:
                    qualifications_list.append(i)

            # create job_description word list using regex to isolate terms
            description_list = [i.lower() for i in re.split('\s|(?<!\d)[,.]|[,.](?!\d)', job_description)]

            # append benefits key words to benefits list
            for i in benefits_keyword:
                if i in description_list:
                    benefits_list.append(i)

            # append qualifications key words to qualifications list 
            for i in qualifications_keyword:
                    if i in description_list:
                        qualifications_list.append(i)

            # add scraped jobs data to 'jobs' dictioanry created above
            jobs.append({"Company_Name_Rating": company_name_rating, 
                        "Job_Title": job_title,
                        "Location": location,
                        "Salary_Estimate": salary_estimate,
                        "Company_Size": company_size,
                        "Company_Type": company_type,
                        "Company_Sector": company_sector,
                        "Year_Founded": year_founded,
                        "Company_Industry": company_industry,
                        "Company_Revenue": company_revenue,
                        "Job_Benefits": benefits_list,
                        "Job_Qualifications": qualifications_list})
            
            # after each job scrape, print page_count, job count for that page, and overall scrape count for entire scrape
            if page_count == 1:
                print("Page:", page_count, "Count:", count, "Total:", count, job_title)
            if page_count > 1:
                print("Page:", page_count, "Count:", count, "Total:", (count+((page_count*30)-30)), job_title)
            
            # reset lists and increase count for next iteration of the while loop
            benefits_list = []
            qualifications_list = []
            count += 1
            count_str = str(count)

    except ElementClickInterceptedException:
        pass
    
    # once done with an entire page, print done with page_count and reset count and increase page_count by 1
    print(f"Done with page {page_count}")
    page_count += 1
    count = 1
    count_str = str(count)
    
    # click on next page to scrape
    try:
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"li[class='job-search-key-106v8g7 e1gri00l2']"))).click()
        time.sleep(random.randint(4,7))
    except TimeoutException:
            print("\n *****SCRAPING TERMINATED*****")  # terminate scrape if no more pages to scrape
            break

Page: 1 Count: 1 Total: 1 Principal, Data Scientist ProServe
Page: 1 Count: 2 Total: 2 Sr. Data & ML Engineer
Page: 1 Count: 3 Total: 3 Senior Data Analyst
Page: 1 Count: 4 Total: 4 Data Scientist - ProServe
Page: 1 Count: 5 Total: 5 Senior Director, Risk Management Technology, Data Office
Page: 1 Count: 6 Total: 6 Senior Data Scientist
Page: 1 Count: 7 Total: 7 Senior Data Scientist - Nationwide Opportunities
Page: 1 Count: 8 Total: 8 Lead AI/Machine Learning Engineer
Page: 1 Count: 9 Total: 9 Director-Risk Management Technology-Data Office
Page: 1 Count: 10 Total: 10 Sr. Data Science Engineer, VMware Cloud on AWS - Opportunity for Working Remotely Burlington, VT
Page: 1 Count: 11 Total: 11 Dev Ops Engineer AI/ML
Page: 1 Count: 12 Total: 12 AWS Machine Learning (ML) Engineer
Page: 1 Count: 13 Total: 13 Sr Data Scientist
Page: 1 Count: 14 Total: 14 Data Scientist for Fulfillment
Page: 1 Count: 15 Total: 15 Senior Machine Learning Engineer
Page: 1 Count: 16 Total: 16 Data Scientist / Ma

In [11]:
len(jobs)    # check length of jobs as a sanity check to make sure it makes sense

17

In [12]:
# turn dictionary object into a pandas dataframe
df = pd.DataFrame(jobs) 
df.sample(5)

,Company_Name_Rating,Job_Title,Location,Salary_Estimate,Company_Size,Company_Type,Company_Sector,Year_Founded,Company_Industry,Company_Revenue,Job_Benefits,Job_Qualifications
1,"Amazon Web Services, Inc.\n3.8",Sr. Data & ML Engineer,Vermont,Employer Provided Salary:$122K,10000+ Employees,Company - Public,Information Technology,1994,Internet,$10+ billion (USD),[remote],"[computer science, deep learning, machine lear..."
3,"Amazon Web Services, Inc.\n3.8",Data Scientist - ProServe,Vermont,Employer Provided Salary:$115K,10000+ Employees,Company - Public,Information Technology,1994,Internet,$10+ billion (USD),[remote],"[scikit-learn, computer science, natural langu..."
9,VMware\n4.3,"Sr. Data Science Engineer, VMware Cloud on AWS...","Burlington, VT",$68K - $157K (Glassdoor est.),10000+ Employees,Company - Public,Information Technology,1998,Computer Hardware & Software,$5 to $10 billion (USD),[],"[computer science, sql, mathematics, engineeri..."
15,"Attadale Partners, LLC",Data Scientist / Machine Learning Engineer (NLP),United States,-1,-1,-1,-1,-1,-1,-1,"[paid time off, health savings account, health...","[computer science, natural language processing..."
10,Change Healthcare\n3.3,Dev Ops Engineer AI/ML,Vermont,-1,-1,-1,-1,-1,-1,-1,[],"[computer science, deep learning, machine lear..."


In [13]:
# export the dataframe to a csv 
df.to_csv('data/glassdoor_data.csv', index=False)